In [19]:
import zipfile
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
import numpy as np
from sklearn import linear_model, metrics, model_selection
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split



In [20]:
zip = zipfile.ZipFile("data/houses.zip")
zip.extractall("data")
zip.close()

In [21]:
houses = pd.read_csv("data/houses.csv")

In [17]:
# Esta vez, nuestro modelo serán el 80% de los datos y el test el 20% de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = , random_state = )

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=, random_state=)
for train_index, test_index in split.split():
    X_train = .iloc[train_index]
    X_test = .iloc[test_index]

In [ ]:
# Crea una nueva regresión lineal (no uses el mismo nombre que la anterior!!!) usando X_train e y_train
# Realiza la regresión lineal

# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
#X_train = USA_Housing[["Avg. Area Income",	"Avg. Area House Age",	"Avg. Area Number of Rooms", 	"Avg. Area Number of Bedrooms",	"Area Population"]]

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo_ = sm.OLS(endog=y_train, exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo_ = modelo_.fit()
print(modelo_.summary())

In [ ]:
model = LinearRegression()
model.fit(X, y)

In [ ]:
model_r = Ridge()
model_r.fit(X, y)

In [ ]:
model_l = Lasso()
model.l.fit(X, y)

In [ ]:
model_e = ElasticNet(alpha=0.1, l1_ratio=1)
model_e.fit(X, y)